# Integrals for the Hartree-Fock Approximation
<br></br>

## Nomenclature

<p>
    <br/>
    
Within this python script, the wave functions, $\psi$, that represent atomic wavefunctions are each composed of a set of contracted guassians that are added lineary and multiplied by respective contraction coeffiecents to produce the wavefunction $\psi$. A guassian is represented as $e^{-\alpha|R_A|^2}$.  When $\psi_1$ and $\psi_2$ are being integrated, $\int\psi_1\psi_2d\tau$, the resulting integral is the multiplication of two guassians, such that the integration reduces to: $$[\dfrac{\pi}{(\alpha+\beta)}]^{\dfrac{3}{2}}\cdot e^{\dfrac{-\alpha\beta}{\alpha+\beta}\cdot|R_a-R_b|^2}$$

The rest of this description will focus upon explaining the meaning behind the nomenclature used in variable naming in this script in regrads to the above analytical expansion of $\int\psi_1\psi_2d\tau$.

$\alpha$ and $\beta$ both refer to the orbital coefficent of a guassian that represents how widespread or narrow the guassian will be. The sum of $\alpha+\beta$ is $p$, while the product of $\alpha\cdot\beta$ is $ab$. 
    
Located on Page 412 of Szabo.  
</p>

In [1]:
import math
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## Overlap Integral

<p> 
    <br/>
    The overlap integral describes the amount of overlap bwetween two atomic orbitals. The value ranges from 1 to 0, with zero meaning there is no overlap between the orbitals, and 1 meaning the orbitals are the same and overlap exactly. 
    <br/> 
    The overlap integral is computed as follows: 
    $$\int \Psi^*_A\Psi_B  d\tau $$
    Reference is Page 412 of Szabo
</p>

In [2]:
def overlap(molecule):

    #init empty list for the overlap matrix
    S = [] 
    
	#iterate thorugh all atoms	
    for index1, atom1 in enumerate(molecule.atomData):
        S.append([])

        for index2, atom2 in enumerate(molecule.atomData):
            S[index1].append(0)			
            
            #prepare basis sets 
            psi1 = atom1.basisSet
            psi2 = atom2.basisSet

            #overlap integral is simply the integral of basis function 1 times basis function 2
            #is done with guassians thorough guassian multiplication, which results in a new 
            #guassian which is then integrated analytically, 
            #and whose analytical equation is solved here for the overlap 
            
            #iterate through all contracted guassians that compose psi1 and psi 2
            for cg1 in psi1.contractedGuassians:
                for cg2 in psi2.contractedGuassians:
                    
                    #get overlap contracted guassian 
                    cg3 = cg1.multiply(cg2)
                
                    #get constant to multiply overlap integral by 
                    constant = cg3.constant * cg1.contraction * cg2.contraction
                 
                    #compute overlap coefficent
                    S[index1][index2] += constant * pow((math.pi/cg3.orbitalExponet), 3/2) 	

    return S

## Kinetic Energy Integral
<p>
    <br />
    This integral computes the kinetic energy of the electrons, which is used in forming the electronic hamiltonian. 
    <br />
    The integral is as follows:
    $$ \int \psi^*_A  (-\frac{1}{2}\nabla^2) \psi_B d\tau $$
    Located on page 427 of Szabo.
</p>


In [3]:
def kineticEnergy(molecule):
    
    #init empty list for the kinetic energy matrix
    T = []
	
	#iterate thorugh all atoms	
    for index1, atom1 in enumerate(molecule.atomData):
        T.append([])
        for index2, atom2 in enumerate(molecule.atomData):
            T[index1].append(0)			
            
            #prepare basis sets 
            psi1 = atom1.basisSet
            psi2 = atom2.basisSet  
            
            #KE integral computed by taking guassian 1 times -1/2 del squared acting upon guassian 2
            #equation located on page 427 of Szabo
            
            #iterate through all the contracted guassians of psi1 and psi2 
            for cg1 in psi1.contractedGuassians:
                for cg2 in psi2.contractedGuassians:
                
                    ab = cg1.orbitalExponet * cg2.orbitalExponet
                    p = cg1.orbitalExponet + cg2.orbitalExponet
                    c1 = ab / p 
                    c2 = pow(math.pi / p, 3/2)
                    distanceSquared = pow((cg1.coord - cg2.coord).magnitude(), 2)
                    e = math.exp(-c1*distanceSquared)
                    constant = cg1.contraction * cg2.contraction * cg1.constant * cg2.constant
                   
                    T[index1][index2] += constant * c1 * (3 - (2*c1*distanceSquared) ) * c2 * e
    
    return T

## Error Function Definition
<p>
    <br/>
    The error function is defined below to be used in the Nuclear-Electron Attraction and Electron-Electron repulsion integrals.
</p>

In [4]:
def errorFunction(errorInput, distance):
    
    if(distance < 10**-50):
         return 1 - (errorInput/3)
    else:
         return 0.5 * pow(math.pi/errorInput, 0.5) * math.erf(pow(errorInput, 0.5))
    

## Nuclear-Electron Attraction Integral
<p>
    <br />
    The nuclear-eclectron attraction integral computes the potential energy surface of the molecule in terms of the attractive forces between the nucli and the electrons. This integral is used to construct the electronic hamiltonian in conjunction with the kinetic energy integral. The final solution to the nuclear-electron attraction is approximated with the error function with an combined with an artificial asymptote at $R=0$.
    <br/>
    The integral is as follows:
    $$ \int \psi^*_A (-\frac{Z_c}{r_{c}}) \psi_B d\tau $$
    Located on page 429 of Szabo.
</p>

In [5]:
def nuclearAttraction(molecule):
    
    #init nuclear attraction matrix 
    #each nuclear attraction list is composed of 1 matrix for each atom in the molecule, which includes a psi x psi matrix of values from the primative guassian compuatation 
    V = []
    
    #used for graphing purposes
    distances = []
    errorInputs = []
    errorValues = []
    
    #iterate through all atoms for Z
    for atomIndex, atom in enumerate(molecule.atomData):
        
        V.append([])

        #prepare graph lists
        distances.append([])
        errorInputs.append([])
        errorValues.append([])

        #iterate through all the atoms present and availible
        for index1, atom1 in enumerate(molecule.atomData):
            V[atomIndex].append([])
            
            distances[atomIndex].append([])
            errorInputs[atomIndex].append([])
            errorValues[atomIndex].append([])
            
            for index2, atom2 in enumerate(molecule.atomData):
                V[atomIndex][index1].append(0)
                
                distances[atomIndex][index1].append([])
                errorInputs[atomIndex][index1].append([])
                errorValues[atomIndex][index1].append([])
                
                #prepare basis sets
                psi1 = atom1.basisSet
                psi2 = atom2.basisSet
            
                #iterate through all the primative guassians
                for cg1 in psi1.contractedGuassians:
                    for cg2 in psi2.contractedGuassians:
                   
                        #compute data needed for the integral
                        ab = cg1.orbitalExponet * cg2.orbitalExponet
                        p = cg1.orbitalExponet + cg2.orbitalExponet
                        e = (-ab/p) *  pow((cg1.coord - cg2.coord).magnitude(), 2)
                        c1 = (-2 * math.pi) / p
                        cg3 = cg1.multiply(cg2)
                                             
                        constant = cg1.contraction * cg2.contraction * c1 * atom.Z * math.exp(e)
                        errorInput = p * math.pow((cg3.coord - atom.coord).magnitude(), 2)
                    
                        #error input cutoff and else equation from Szabo pg. 437
                        #use first error equation to simulate asymptote for very small values
                        #but, due to numerical errors with python, small numbers only occurs if index1==index2, but sometimes, such values may be larger, resulting in a negative error that is remedied by the second error equation
                        electronNuclearDistance = (cg3.coord - atom.coord).magnitude()
                        error = errorFunction(errorInput, electronNuclearDistance)
                        #if(electronNuclearDistance < 10**-30):
                         #   error = 1 - (errorInput/3)
                        #else:
                        #    error = 0.5 * pow(math.pi/errorInput, 0.5) * math.erf(pow(errorInput, 0.5))
                    
                        distances[atomIndex][index1][index2].append(electronNuclearDistance)
                        errorInputs[atomIndex][index1][index2].append(errorInput)
                        errorValues[atomIndex][index1][index2].append(error)
                        #distances[atomIndex].append(electronNuclearDistance)
                        #errorInputs[atomIndex].append(errorInput)
                        #errorValues[atomIndex].append(error)
                        
                        V[atomIndex][index1][index2] += constant * error * cg1.constant * cg2.constant 

    
    #fig = plt.figure()
    #ax = fig.add_subplot(111, projection='3d')
    #ax.scatter(errorInputs[0][0][0], distances[0][0][0], errorValues[0][0][0], label = "A1P1P1")
    #ax.scatter(errorInputs[1][0][1], distances[1][0][1], errorValues[1][0][1], label = "A1P1P2")
    #ax.scatter(errorInputs[0][1][1], distances[0][1][1], errorValues[0][1][1], label = "A1P2P2")
    #ax.legend()
    #ax.scatter(distances[0], errorInputs[0], errorValues[0])
    plt.plot(distances[0][0][0], errorValues[0][0][0], 'rs', label="A1e1e1")
    plt.plot(distances[0][1][0], errorValues[0][1][0], 'gd', label="A1e2e1")
    plt.plot(distances[0][1][1], errorValues[0][1][1], 'bp', label="A1H2H2")
   # plt.plot(distances[0][0][1], errorValues[0][0][1], 'b^', label="Error Input")
   # plt.plot(errorInputs[], errorInputs[1], 'r^', label="Error Input")
    plt.legend()
    
    return V

## Electron-Electron Repulsion Integral

<p>
    <br/>
    This integral computes the energy that each electrons feels towards all other electrons near it. Also called the four center integral because it involves integrating over four wavefunctions simultaneously.
    <br/>
    The integral is as follows: $$\int\psi^*_{A1}\psi^*_{B1}(\dfrac{1}{r_{12}})\psi_{C2}\psi_{D3} d\tau_1d\tau_2$$
</p>

In [6]:
def electronElectronRepulsion(molecule):
    
    #init electron-electron repulsion list 
    electronRepulsion = []

    #iterate through PsiA1PsiA2 and PsiB1PsiB2
    for index1, atom1 in enumerate(molecule.atomData):
        electronRepulsion.append([])
        for index2, atom2 in enumerate(molecule.atomData):
            electronRepulsion[index1].append([])
            
            #prepare atomic basis sets
            psi1 = atom1.basisSet
            psi2 = atom2.basisSet
            
            for index3, atom3 in enumerate(molecule.atomData):
                electronRepulsion[index1][index2].append([])
                for index4, atom4 in enumerate(molecule.atomData):
                    electronRepulsion[index1][index2][index3].append(0)
                    
                    #prepare atomic basissets
                    psi3 = atom3.basisSet
                    psi4 = atom4.basisSet

                    #iterate through all of the primative guassians for each of the wavefunctions
                    for cg1 in psi1.contractedGuassians:
                        for cg2 in psi2.contractedGuassians:
                            for cg3 in psi3.contractedGuassians:
                                for cg4 in psi4.contractedGuassians:

                                    pA = cg1.orbitalExponet + cg2.orbitalExponet
                                    pB = cg3.orbitalExponet + cg4.orbitalExponet
                                    abA = cg1.orbitalExponet * cg2.orbitalExponet
                                    abB = cg3.orbitalExponet * cg4.orbitalExponet
                                    
                                    pAB = pA + pB

                                    term1 = (2 * pow(math.pi, 5/2)) / (pA*pB*pow(pAB, 1/2))
                                    distanceAB = pow( (cg1.coord - cg2.coord).magnitude(), 2)
                                    distanceCD = pow( (cg3.coord - cg4.coord).magnitude(), 2)
                                    e = ((-abA/pA) * distanceAB) - ( (abB/pB) * distanceCD )
                                    cgAB = cg1.multiply(cg2)
                                    cgCD = cg3.multiply(cg4)
                                    distanceABCD = (cgAB.coord - cgCD.coord).magnitude()
                                    errorInput = ( (pA * pB) / pAB) * pow(distanceABCD, 2)
                                    error = errorFunction(errorInput, distanceABCD)
        
                                    constant = cg1.constant * cg2.constant * cg3.constant * cg4.constant * cg1.contraction * cg2.contraction * cg3.contraction * cg4.contraction
                                   
                                    electronRepulsion[index1][index2][index3][index4] += term1 * math.exp(e) * error * constant
    return electronRepulsion                    